<div class="contentcontainer med left" style="margin-left: -50px;">
<dl class="dl-horizontal">
  <dt>Description</dt> <dd> Streams quickstart guide</dd>
  <dt>Author</dt> <dd>Jean-Luc Stevens</dd>
  <dt>HoloViews</dt> <dd>>1.6.2</dd>
  <dt>Python</dt> <dd>2.7/3.3+</dd>
</dl>
</div>

This quickstart is designed to introduce the basics of the streams system planned for HoloViews 1.7. A stream is a simple object with a set of parameters that allow events to affect a displayable HoloViews object. Starting with a simple function to generate NumPy arrays, this quickstart guide serves as a short introduction to the ``Image`` element, ``DynamicMap`` and finally an example of how streams are used. This notebook uses core HoloViews together with the matplotlib backend.

In [ ]:
import holoviews as hv
import numpy as np
from holoviews.streams import Stream
hv.notebook_extension()

# Simple NumPy arrays

The following function generates 2D NumPy arrays that sample a sinusoidal pattern:

In [ ]:
x,y = np.mgrid[-50:51, -50:51] * 0.1

def sine_array(phase, freq):
    return np.sin(phase + (freq*x**2+freq*y**2))

Here is an example array generated using ``phase=0`` and ``freq=1``. We can see it is a ``numpy.ndarray`` with a shape of ``(101,101)``.

In [ ]:
arr = sine_array(0,1)
print(type(arr))
print(arr.shape)

# ``hv.Image``

The following function generates an array with ``sine_array`` and passes is to a ``hv.Image`` element:

In [ ]:
def sine_image(phase, freq):
    return hv.Image(sine_array(phase, freq))

Here is an example of the output:

In [ ]:
sine_image(0,1)

And as with all HoloViews elements, you can use the ``+`` operator to generate a layout:

In [ ]:
sine_image(0,1) + sine_image(0.5,2)

For more information about HoloViews elements such as ``Image``, see the [Elements tutorial](http://holoviews.org/Tutorials/Elements.html). For more information about the ``+`` operator and ``Layout`` objects, see the [Containers tutorial](http://holoviews.org/Tutorials/Containers.html).

# DynamicMap

We can use our ``sin_image`` function in a HoloViews ``DynamicMap`` to interactively explore this phase and frequency space by declaring ``phase`` and ``frequency`` dimensions with appropriate ranges. Note that this example and those that follow need a live notebook kernel to work such as [this one on mybinder](http://mybinder.org/repo/ioam/holoviews-contrib/notebooks/notebooks/quickstart/Streams.ipynb):

In [ ]:
hv.DynamicMap(sine_image, kdims=[hv.Dimension('phase',range=(0, np.pi)),
                                 hv.Dimension('frequency', range=(0.01,np.pi))])

A DynamicMap takes a callable (such as a function) that returns HoloViews elements and uses it to dynamically update a visualization. In the example above, this callable receives the ``phase`` and ``frequency`` values as positional arguments as specified by the two numeric sliders. For more information about ``DynamicMap``, see the [DynamicMap tutorial](http://holoviews.org/Tutorials/Dynamic_Map.html).

## Streams

Streams are a new type of object introduced after the HoloViews 1.6.2 release. Many streams are ready to import and use directly but in this quickstart we will create a custom stream by subclassing from ``Stream``:

In [ ]:
import param
from holoviews.streams import Stream

class SineParameters(Stream):
    
    phase = param.Number(default=0)
    
    freq = param.Number(default=0.01)

All that is necessary is to declare parameters using the [param library](http://ioam.github.io/param/). This is not a special property of Streams as almost all HoloViews objects are parametrized in this way. 

We can how make instances of this stream type:

In [ ]:
sine_params = SineParameters() 

print('Initial Phase: %s' % sine_params.phase)
print('Initial Frequency: %s' % sine_params.freq)

As you can see, this instance uses the parameter defaults defined in the class definition above. We can also supply the values we want to initialize the stream with:

In [ ]:
sine_params = SineParameters(phase=0.5, freq=0.1)

print('Initial Phase: %s' % sine_params.phase)
print('Initial Frequency: %s' % sine_params.freq)

What streams do is to supply an ``update`` method to update these parameter values which will also update the visualization in the notebook as we will see in the next section:

In [ ]:
sine_params.update(phase=0.3, freq=1)

print('Updated Phase: %s' % sine_params.phase)
print('Updated Frequency: %s' % sine_params.freq)

# Streams and DynamicMap

To use streams, you supply them to a ``DynamicMap`` using the ``streams`` argument instead of the ``kdims`` which were used to generate the slider example above:

In [ ]:
hv.DynamicMap(sine_image, kdims=[], streams=[sine_params])

The stream now lets us control the phase and frequency of the visualization above instead of using sliders. Running the following cell will double the frequency of the sinusoidal rings shown above:

In [ ]:
sine_params.update(freq=2)

We did not need to change the ``sine_image`` callback - whereas the arguments were passed in by position when ``kdims`` were used (see the sliders example above) they are now passed by name via keyword arguments.

Now we will update the frequency of the plot above using a simple loop:

In [ ]:
for freq in np.linspace(0.1, 2):
    sine_params.update(freq=freq)

Of course we can also update the phase to $\frac{\pi}{2}$:

In [ ]:
sine_params.update(freq=2, phase=np.pi/2)

And then set it back to zero phase:

In [ ]:
sine_params.update(freq=2, phase=0)

Now lets update the phase in a loop - watch the plot animate above!

In [ ]:
phase = 0
for i in range(200):
    phase += 0.3
    sine_params.update(phase=phase)

``DynamicMap`` objects with streams act like any other HoloViews objects - you can compose them in Layouts with ``+`` and use the option system too:

In [ ]:
%%opts Image (cmap='viridis')
hv.DynamicMap(sine_image, kdims=[], streams=[sine_params]) + sine_image(0,1)

Note that when you update the phase in a loop, both the visualization above and the one before it update as both of the ``DynamicMap`` object involves use the same stream:

In [ ]:
phase = 0
for i in range(50):
    phase += 0.3
    sine_params.update(phase=phase)

You will notice that the visualizations update much slower - not only is the layout a more complex plot but matplotlib is now re-rendering two sets of plots at once and the results have to be communicated to the browser. For many elements, you will get get better performance using the [Bokeh backend](http://holoviews.org/Tutorials/Bokeh_Elements.html) and for more information about the options system, see the [Options tutorial](http://holoviews.org/Tutorials/Options.html).